# Tutorial on `opr.inference.pipelines.localization` subpackage

This tutorial demonstrates the new `LocalizationPipeline` that combines top‑k Place Recognition with point cloud Registration.

You will:
- Create a tiny set of database point clouds (`.bin` files with [x,y,z]).
- Define a minimal fake Place Recognition output (two candidates).
- Run `LocalizationPipeline` to estimate poses for each candidate and select the best.

Requirements:
- `numpy`, `torch` (no FAISS/Open3D needed for this minimal example).


In [1]:
# Setup tiny DB and stubs
from pathlib import Path
import numpy as np
import torch

from opr.inference.pipelines.localization import LocalizationPipeline
from opr.inference.pipelines.place_recognition import PlaceRecognitionResult

base = Path("./_demo_localization").resolve()
(base / "scans").mkdir(parents=True, exist_ok=True)

# Two small DB point clouds
pc_a = np.array([[0.0, 0.0, 0.0], [1.0, 0.0, 0.0]], dtype=np.float32)
pc_b = np.array([[0.0, 0.0, 0.0], [0.0, 1.0, 0.0]], dtype=np.float32)
(pc_a).tofile(base / "scans" / "000100.bin")
(pc_b).tofile(base / "scans" / "000101.bin")

db_idx = np.array([100, 101], dtype=np.int64)
db_pose = np.array(
    [
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
        [1.0, 2.0, 3.0, 0.0, 0.0, 0.0, 1.0],
    ],
    dtype=np.float64,
)
db_pc_path = np.array(["scans/000100.bin", "scans/000101.bin"], dtype=object)

class FakeIndex:
    def get_meta(self, rows: np.ndarray):
        return db_idx[rows], db_pose[rows], db_pc_path[rows]

class FakePR:
    def infer(self, _input: dict, k: int) -> PlaceRecognitionResult:
        return PlaceRecognitionResult(
            descriptor=np.zeros(8, dtype=np.float32),
            indices=np.array([0, 1], dtype=np.int64),
            distances=np.array([0.5, 1.0], dtype=np.float32),
            db_idx=db_idx,
            db_pose=db_pose,
        )

class FakeReg:
    def infer(self, query_pc: torch.Tensor, db_pc: torch.Tensor) -> np.ndarray:
        # Identity transform: estimated pose equals db_pose
        return np.eye(4, dtype=np.float64)

pipeline = LocalizationPipeline(
    index=FakeIndex(),
    place_recognition=FakePR(),
    registration=FakeReg(),
    index_root=base,
)

query_pc = torch.tensor([[0.0, 0.0, 0.0], [0.5, 0.0, 0.0]], dtype=torch.float32)
result = pipeline.infer(pr_input={}, query_pc=query_pc, k=2)

print(f"Chosen idx: {result.chosen_idx}")
for c in result.candidates:
    print(f"Candidate: {c.idx} pr_distance: {c.pr_distance} est_pose: {c.estimated_pose}")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Chosen idx: 100
Candidate: 100 pr_distance: 0.5 est_pose: [0. 0. 0. 0. 0. 0. 1.]
Candidate: 101 pr_distance: 1.0 est_pose: [1. 2. 3. 0. 0. 0. 1.]
